<a href="https://colab.research.google.com/github/pninav/Cycle_Gan/blob/main/Cycle_GAN_Eval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Mounting to google-drive

In [ ]:
from google.colab import drive
import os
drive.mount('/content/Pnina/')
os.chdir('/content/Pnina/MyDrive/')

Mounted at /content/Pnina/


Cloning CycleGan project (creates a local directory in google drive, initializes a GitHub directory inside it), and installing requirements. 

In [ ]:
!rm -rf /content/Pnina/MyDrive/Cycle_Gan
!git clone https://github.com/pninav/Cycle_Gan.git
os.chdir('/content/Pnina/MyDrive/Cycle_Gan')
!pip install -r requirements.txt  

Download Kaggle dataset using Kaggle API.

In [ ]:
import os
os.chdir('/content/Pnina/MyDrive/Cycle_Gan/')
os.environ['KAGGLE_CONFIG_DIR'] = '/content/Pnina/MyDrive/Cycle_Gan/kaggle/'
!mkdir -p kaggle_dataset
os.chdir('/content/Pnina/MyDrive/Cycle_Gan/kaggle_dataset')
!kaggle competitions download -c gan-getting-started 

Create Cycle_Gan folder (the GitHub files will be transferred there)

In [ ]:
os.chdir('/content/Pnina/MyDrive/Cycle_Gan/')


Run the train.py notebook - Train the CycleGAN model.

In [ ]:
!python train.py --cuda --eval

Create output & run folders
Download the training weights

In [ ]:
import os
os.chdir('/content/Pnina/MyDrive/Cycle_Gan/')
!mkdir -p output
os.chdir('/content/Pnina/MyDrive/Cycle_Gan/output')
!gdown --id 1PLazQWL6YmA8l-W-gEUib2aGSKfRyWgH
!gdown --id 1d7on629bJcAsUvi8Pd5j-I95UhrxxN30
os.chdir('/content/Pnina/MyDrive/Cycle_Gan/')
!mkdir -p runs 
os.chdir('/content/Pnina/MyDrive/Cycle_Gan/runs')
!mkdir -p best 
os.chdir('/content/Pnina/MyDrive/Cycle_Gan/runs/best')
!gdown --id 1rhrDBBOostvxDr39MMk3kY7tHzDi8Ptf



Run the test.py notebook While displaying a progress bar for each iteration

In [ ]:
os.chdir('/content/Pnina/MyDrive/Cycle_Gan/')
!python test.py --cuda

View sample Images.

In [ ]:
from google.colab.patches import cv2_imshow
from glob import glob
import cv2

for file in glob('/content/Pnina/MyDrive/Cycle_Gan/output/B_reduced2/*.*'):
  img = cv2.imread(file, cv2.IMREAD_UNCHANGED)
  cv2_imshow(img) 

Create an output file (zip file) containing approximately 7000 Monet-style images (the input photos in Monet style).

---



In [ ]:
!mkdir /content/Pnina/MyDrive/Cycle_Gan/output/submission
!zip -r  /content/Pnina/MyDrive/Cycle_Gan/output/submission/Pnina.zip /content/Pnina/MyDrive/Cycle_Gan/output/B

Using TensorBoard - view dashboards such as scalars, graphs, histograms, and others results.

In [ ]:
%load_ext tensorboard
%tensorboard --logdir=/content/Pnina/MyDrive/Cycle_Gan/runs  --port 6006


Downloaded a pre-trained model on ImageNet of Inception v3 from http://download.tensorflow.org/models/image/imagenet/inception-2015-12-05.tgz  


In [ ]:
!wget http://download.tensorflow.org/models/image/imagenet/inception-2015-12-05.tgz

Extract the zip of the inception, especially the weights (classify_image_graph_def.pb).

In [ ]:
os.chdir('/content/Pnina/MyDrive/Cycle_Gan')
!tar zxvf /content/Pnina/MyDrive/Cycle_Gan/inception-2015-12-05.tgz

In [ ]:
!pip install tensorflow==1.15
!pip3 install protobuf==3.6.0


You must restart the runtime in order to use newly installed versions
A button will be displayed to confirm a restart to upload an advanced version of TensorFlow.


Initialize parameters for MiFID:

  I.	MODEL_PATH: inception weights

  II.	TRAIN_DIR:  thirty reduced Monet images we selected for training
  
  III.	OUT_DIR:  thirty Monet style output images


In [ ]:
import os
from pathlib import Path
import shutil

import numpy as np
import torch
from torch import nn, optim
import torch.nn.functional as F
from torchvision import datasets, transforms
from torchvision.utils import save_image
MODEL_PATH = '/content/Pnina/MyDrive/Cycle_Gan/classify_image_graph_def.pb'
TRAIN_DIR = Path('/content/Pnina/MyDrive/Cycle_Gan/kaggle_dataset/monet_reduced' )
OUT_DIR = Path('/content/Pnina/MyDrive/Cycle_Gan/output/B_reduced')
OUT_DIR.mkdir(exist_ok=True)


MiFID - evaluation method

In [ ]:
from dataclasses import dataclass
from pathlib import Path
import warnings

import numpy as np
from PIL import Image
from scipy import linalg
import tensorflow as tf


class KernelEvalException(Exception):
    pass


@dataclass
class MiFIDEvaluator(object):
    model_path: str
    train_images_path: str
    feature_path: str = None
    imsize: int = 256
    output_layer: str = 'Pretrained_Net/pool_3:0'
    input_layer: str = 'Pretrained_Net/ExpandDims:0'
    output_shape: int = 2048
    cosine_distance_eps: float = 0.1
    batch_size: int = 1
    fid_epsilon: float = 1e-14
    
    def __post_init__(self):
        tf.reset_default_graph()
        self.create_model_graph()
        with tf.Session() as sess:
            if self.feature_path is None:
                self.mu2, self.sigma2, self.features2 = self._handle_path_memorization(
                    self.train_images_path, sess, is_checksize=False, is_check_png=False)
            else:
                with np.load(self.feature_path) as f:
                    self.mu2, self.sigma2, self.features2 = f['m'], f['s'], f['features']
    
    def create_model_graph(self):
        with tf.gfile.FastGFile(self.model_path, 'rb') as f:
            graph_def = tf.GraphDef()
            graph_def.ParseFromString(f.read())
            _ = tf.import_graph_def(graph_def, name='Pretrained_Net')
            
    def img_read_checks(self, filename, is_checksize=False, is_check_png=False,check_imsize = 256):
        im = Image.open(str(filename)).convert('RGB')
        if is_checksize and im.size != (self.imsize, self.imsize):
            raise KernelEvalException(f'The images are not of size {check_imsize}')
    

        if self.imsize is None:
            return im
        else:
            return im.resize((self.imsize, self.imsize), Image.ANTIALIAS)
        
    def _get_model_layer(self, sess):
        layer = sess.graph.get_tensor_by_name(self.output_layer)
        ops = layer.graph.get_operations()
        for op_idx, op in enumerate(ops):
            for o in op.outputs:
                shape = o.get_shape()
                if shape._dims != []:
                    shape = [s.value for s in shape]
                    new_shape = []
                    for j, s in enumerate(shape):
                        if s == 1 and j == 0:
                            new_shape.append(None)
                        else:
                            new_shape.append(s)
                    o.__dict__['_shape_val'] = tf.TensorShape(new_shape)
        return layer
        
    def get_activations(self, images, sess):
        inception_layer = self._get_model_layer(sess)
        n_images = images.shape[0]
        if self.batch_size > n_images:
            warnings.warn('batch size is bigger than the data size. setting batch size to data size')
            self.batch_size = n_images
        print("NUmber of Images",n_images)
        n_batches = n_images // self.batch_size + 1
        pred_arr = np.empty((n_images, self.output_shape))
        for i in range(n_batches):
            start = i * self.batch_size
            if start + self.batch_size < n_images:
                end = start + self.batch_size
            else:
                end = n_images

            batch = images[start:end]
            pred = sess.run(inception_layer, {self.input_layer: batch})
            pred_arr[start:end] = pred.reshape(-1, self.output_shape)
        return pred_arr
        
    def calculate_activation_statistics(self, images, sess):
        act = self.get_activations(images, sess)
        mu = np.mean(act, axis=0)
        sigma = np.cov(act, rowvar=False)
        return mu, sigma, act
            
    def _handle_path_memorization(self, path, sess, is_checksize, is_check_png):
        path = Path(path)
        files = list(path.glob('*.jpg')) + list(path.glob('*.png'))

        # In production we don't resize input images. This is just for demo purpose. 
        x = np.array([np.array(self.img_read_checks(fn, is_checksize, is_check_png)) for fn in files])
        m, s, features = self.calculate_activation_statistics(x, sess)
        del x
        return m, s, features
    
    def calculate_frechet_distance(self, mu1, sigma1):
        mu1 = np.atleast_1d(mu1)
        mu2 = np.atleast_1d(self.mu2)
        sigma1 = np.atleast_2d(sigma1)
        sigma2 = np.atleast_2d(self.sigma2)

        assert mu1.shape == mu2.shape, 'Training and test mean vectors have different lengths'
        assert sigma1.shape == sigma2.shape, 'Training and test covariances have different dimensions'

        # product might be almost singular
        covmean, _ = linalg.sqrtm(sigma1.dot(sigma2), disp=False)
        if not np.isfinite(covmean).all():
            msg = f'fid calculation produces singular product; adding {self.eps} to diagonal of cov estimates'
            warnings.warn(msg)
            offset = np.eye(sigma1.shape[0]) * self.eps
            covmean = linalg.sqrtm((sigma1 + offset).dot(sigma2 + offset))
            
        # numerical error might give slight imaginary component
        if np.iscomplexobj(covmean):
            if not np.allclose(np.diagonal(covmean).imag, 0, atol=1e-3):
                m = np.max(np.abs(covmean.imag))
                raise ValueError(f'Imaginary component {m}')
            covmean = covmean.real
        tr_covmean = np.trace(covmean)
        return (mu1 - mu2).dot(mu1 - mu2) + np.trace(sigma1) + np.trace(sigma2) - 2 * tr_covmean
    
    @staticmethod
    def normalize_rows(x):
        return np.nan_to_num(x / np.linalg.norm(x, ord=2, axis=1, keepdims=True))
    
    def cosine_distance(self, features1):
        features1_nozero = features1[np.sum(features1, axis=1) != 0]
        features2_nozero = self.features2[np.sum(self.features2, axis=1) != 0]
        norm_f1 = self.normalize_rows(features1_nozero)
        norm_f2 = self.normalize_rows(features2_nozero)

        d = 1.0 - np.abs(np.matmul(norm_f1, norm_f2.T))
        mean_min_d = np.mean(np.min(d, axis=1))
        return mean_min_d
            
    def calculate_kid_given_paths(self, user_images_unzipped_path):
        with tf.Session() as sess:
            sess.run(tf.global_variables_initializer())
            m1, s1, features1 = self._handle_path_memorization(
                user_images_unzipped_path, sess, is_checksize=True, is_check_png=True)

            fid_value = self.calculate_frechet_distance(m1, s1)
            distance = self.cosine_distance(features1)
            return fid_value, distance
        
    def distance_thresholding(self, d):
        if d < self.cosine_distance_eps:
            return d
        else:
            return 1
        
    def evaluate(self, user_images_unzipped_path):
        fid_value, distance = self.calculate_kid_given_paths(user_images_unzipped_path)
        distance = self.distance_thresholding(distance)
        return fid_value, distance, fid_value / (distance + self.fid_epsilon)

Get MiFID score.

In this notebook, I added the functions of calculating the MiFID and activating them to obtain a score for the training model.
It is important to note that the MiFID calculation in this notebook is performed for the training of thirty photo images compared to the thirty photos of Monet.
In contrast, the output of modeling the models on seven thousand images, which I pass on to the competition in Kegel, will get a higher MiFID score because I trained on seven thousand compared to thirty images.

In [ ]:
evaluator = MiFIDEvaluator(MODEL_PATH, TRAIN_DIR)
fid_value, distance, mi_fid_score = evaluator.evaluate(OUT_DIR)
print(f'FID: {fid_value:.5f}')
print(f'distance: {distance:.5f}')
print(f'MiFID: {mi_fid_score:.5f}')